In [256]:
# load the packages needed to run the code
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import re
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
#from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

In [257]:
activity_list = ['Backpacking',
                 'Bird watching',
                 'Camping',
                 'Cross country skiing',
                 'Fishing',
                 'Hiking',
                 'Horseback riding',
                 'Mountain biking',
                 'Nature trips',
                 'Off-road driving',
                 'Paddle sports',
                 'Road biking',
                 'Rock climbing',
                 'Scenic driving',
                 'Skiing',
                 'Snowshoeing',
                 'Surfing',
                 'Running',
                 'Walking']

attraction_list = ['Beach',
                   'Cave',
                   'City walk',
                   'Forest',
                   'Historic site',
                   'Hot springs',
                   'Lake',
                   'Pub walk',
                   'Rails trails',
                   'River',
                   'Views',
                   'Waterfall',
                   'Wild flowers',
                   'Wildlife']

suitability_list = ['Dog friendly',
                    'Kid friendly',
                    'Partially paved',
                    'Paved',
                    'Stroller friendly',
                    'Wheelchair friendly']

In [258]:
# specify the path where you want to save the .csv file
#outpath = '/Users/spk/Documents/Code/Python/webscrape_proj/outputs/banff_national_park_trailforks.csv'
outDIR = '/Users/spk/Documents/GitHub/trail_scraping/outputs/'
if not os.path.exists(outDIR):
    os.mkdir(outDIR)
out_name = 'AllTrails_Kananaskis_CPLU.csv'
# specify the url that you want to scrape from
b_tl_lat=50.90000
b_tl_lng=-115.20000
b_br_lat=50.70000
b_br_lng=-114.70000
url_main = 'https://www.alltrails.com/explore?b_tl_lat=' + str(b_tl_lat) + \
'&b_tl_lng=' + str(b_tl_lng) + \
'&b_br_lat=' + str(b_br_lat) + \
'&b_br_lng=' + str(b_br_lng)

In [259]:
options = webdriver.ChromeOptions()
options = webdriver.ChromeOptions()
#options.add_argument("--headless")
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install(),
                         options=options)
#driver.maximize_window()
#driver.execute_script("document.body.style.zoom='50%'")
timeout = 5

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/spk/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [260]:
driver.get(url_main)
trail_num_xp = '/html/body/div[3]/div[2]/main/div/div/div/div[3]/div[1]/div/div[1]'
trail_table_elem = WebDriverWait(driver, timeout)\
    .until(ec.presence_of_element_located((By.XPATH, 
                                           trail_num_xp)))


In [243]:
trail_num_txt = trail_table_elem.find_element(By.XPATH,
                                         trail_num_xp).text
trail_num_str = trail_num_txt[trail_num_txt.find("(")+1:trail_num_txt.find(")")]
print(trail_num_str + ' trails found. Continue?')


112 trails found. Continue?


In [244]:
if trail_num_str == '1000+':
    print('Too many trails, need to zoom in on map')
else:
    trail_num = int(trail_num_str)

In [245]:
scrollPane = trail_table_elem.find_element(By.XPATH, 
                                           '/html/body/div[3]/div[2]/main/div/div/div/div[3]/div[1]/ul')


In [246]:
viz_trails = scrollPane.find_elements(By.XPATH,
                                      '//*[@id="fullscreen-search-results"]/ul/li')
trail_num_viz = len(viz_trails)

In [247]:
ActionChains(driver).move_to_element_with_offset(scrollPane,
                                                xoffset=5,
                                                yoffset=5).click().perform()

In [248]:
while trail_num_viz < trail_num:
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
    viz_trails = scrollPane.find_elements(By.XPATH,
                                      '//*[@id="fullscreen-search-results"]/ul/li')
    trail_num_viz = len(viz_trails)
    print('\r' + str(trail_num_viz), end='\r', flush=True)


In [250]:
df_out = pd.DataFrame(columns=['Name', 'URL', 'Zone', 'Difficulty', 'Rating', 'Reviews'])
for idx, t in enumerate(viz_trails):
    name = t.find_element(By.XPATH,
                          './/div/a').get_attribute('text')
    print_space = ' ' * (70-len(name))
    print('\r' + str(idx+1) + ' of ' + str(trail_num_viz) + ': ' + name + print_space, 
          end='', flush=True)
    url = t.find_element(By.XPATH,
                      './/div/a').get_attribute('href')
    zone = t.find_element(By.XPATH,
                      './/div/div[2]/a').get_attribute('text')  
    diff = t.find_element(By.XPATH,
                      './/div[2]/div[2]/span[1]').text 
    try:
        rating = re.sub(' Stars', '', t.find_element(By.XPATH,
                                                 './/div[2]/div[2]/span[2]').get_attribute('aria-label'))
    except NoSuchElementException:
        rating = 'NA'
    try:
        revs = int(t.find_element(By.XPATH,
                                  './/div[2]/div[2]/span[3]').text.replace('(', '').replace(')', ''))
    except NoSuchElementException:
        revs = 0
    df_tmp = pd.DataFrame({
        'Name': name,
        'URL': url,
        'Zone': zone,
        'Difficulty': diff,
        'Rating': rating,
        'Reviews': revs
    }, index=[len(df_out.index)])
    df_out = df_out.append(df_tmp)

112 of 112: Mount Glasgow and Mount Cornwall via Little Elbow Trail                                                                 

In [ ]:
for idx, url in enumerate(df_out['URL']):
    name = df_out.loc[idx, 'Name']
    print_space = ' ' * (70-len(name))
    print('\x1b[1K\r' + str(idx+1) + ' of ' + str(trail_num_viz) + ': ' + name + print_space, 
      end='\x1b[1K\r', flush=True)
    driver.get(re.sub('\?ref=result-card', '', re.sub('/explore', '', url)))
    elev_gain = WebDriverWait(driver, timeout)\
    .until(ec.presence_of_element_located(
        (By.XPATH,
         '//*[@id="main"]/div[2]/div[1]/article/section[2]/div/span[2]/span[2]'))).text
    df_out.loc[idx, 'Route type'] = driver.find_element(
        By.XPATH,
        '//*[@id="main"]/div[2]/div[1]/article/section[2]/div/span[3]/span[2]').text
    if bool(re.search(' ft', elev_gain)):
        df_out.loc[idx, 'Elevation_gain_m']  = round(
            float(re.sub(' ft', '', elev_gain).replace(',', '')) * 0.3048, 2)
    else:
        df_out.loc[idx, 'Elevation_gain_m']  = round(
            float(re.sub(' m', '', elev_gain).replace(',', '')) * 0.3048, 2)
    para = driver.find_element(By.XPATH,
                          '//*[@id="auto-overview"]').text.split()
    try:
        df_out.loc[idx, 'Traffic'] = para[para.index('trafficked') - 1]
    except ValueError:
        df_out.loc[idx, 'Traffic'] = 'NA'
    tag_cloud = driver.find_elements(By.XPATH, 
                                     '//*[@id="main"]/div[2]/div[1]/article/section[3]/*')
    activities = []
    attractions = []
    suitabilities = []
    for tag in tag_cloud:
        if tag.text in activity_list:
            activities.append(tag.text)
        if tag.text in attraction_list:
            attractions.append(tag.text)
        if tag.text in suitability_list:
            suitabilities.append(tag.text)
    df_out.loc[idx, 'Activities'] = ', '.join(activities)
    df_out.loc[idx, 'Attractions'] = ', '.join(attractions)
    df_out.loc[idx, 'Suitabilities'] = ', '.join(suitabilities)

In [255]:
df_out.to_csv('/Users/spk/Documents/GitHub/trail_scraping/outputs/AllTrails_csv/' + 
              'AllTrails_attrs_' + str(b_tl_lat) + str(b_tl_lng) + '.csv')